Use the model from https://resourcewatch.org/data/explore/Wat063-Plastic-Input-in-Rivers?section=Discover&selectedCollection=&zoom=11.220883753497942&lat=8.666165796812479&lng=-60.882446428277014&pitch=0&bearing=0&basemap=dark&labels=light&layers=%255B%257B%2522dataset%2522%253A%252238e1bdc7-a843-40b8-ac86-2e47ccd896e4%2522%252C%2522opacity%2522%253A1%252C%2522layer%2522%253A%25228ad07c40-dbb6-4703-8146-a36b9f441dd2%2522%257D%255D&aoi=&page=1&sort=most-viewed&sortDirection=-1 to predict plastic inputs mass/day from rivers.

# Load data

In [ ]:
import geopandas as gpd

In [ ]:
# Model regression parameters
k = 1.85e-3
a = 1.52

In [ ]:
df = gpd.read_file("river_inputs/PlasticRiverInputs.shp")
df.head()

In [ ]:
df.plot()

In [ ]:
df.keys()

# Compute model

Here we compute the plastic mass spilling from points using the model.

In [ ]:
def compute_plastic_mass(mpw, R):
    return (k * mpw * R) ** a

In [ ]:
m_out = compute_plastic_mass(df["mpw"], df["runoff_jan"])
df["m_out"] = m_out

In [ ]:
df.head()